### calculated all the benchmarking metrics

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import anndata as ad
import scanpy as sc

import sys
sys.path.append("./")
import match
import metrics
import utils
from scipy.io import mmread
import os

sys.executable

'/home/bkzhu/python/miniconda3/envs/super_mario_testing/bin/python3'

In [2]:
meta = pd.read_csv('/atac_bench_nrz/10x_e18/data/10x_e18_meta.csv')
annt = meta['celltype'].to_numpy()

In [33]:
# should not reduce scjoint embedding
scjx = pd.read_csv("/atac_bench_nrz/10x_e18/scjoint/raw_labels/rna_embd.csv")
scjy = pd.read_csv("/atac_bench_nrz/10x_e18/scjoint/raw_labels/atac_embd.csv")
scj_dist = utils.cdist_correlation(scjy.to_numpy(), scjx.to_numpy())
scj_full_match, scj_scores = metrics.get_knn_matching(scj_dist)
scjmatch = [np.arange(scjx.shape[0]),scj_full_match,scj_scores]

In [34]:
## maestro use all dims avaliable
msx = pd.read_csv("/atac_bench_nrz/10x_e18/maestro/rna_embed.csv")
msy = pd.read_csv("/atac_bench_nrz/10x_e18/maestro/atac_embed.csv")

ms_dist = utils.cdist_correlation(msy.to_numpy(), msx.to_numpy())
ms_full_match, ms_scores = metrics.get_knn_matching(ms_dist)
msmatch = [np.arange(msx.shape[0]),ms_full_match,ms_scores]

In [52]:
mf = pd.read_csv("/atac_bench_nrz/10x_e18/mf/full_idx_21.csv")
mfmatch = [mf['idx1'].tolist(),mf['idx2'].tolist(),mf['score'].tolist()]

In [36]:
## glue use all avaliable
glx = pd.read_csv("/atac_bench_nrz/10x_e18/glue/e18mouse_rna.csv")
gly = pd.read_csv("/atac_bench_nrz/10x_e18/glue/e18mouse_atac.csv")

gl_dist = utils.cdist_correlation(gly.to_numpy(), glx.to_numpy())
gl_full_match, gl_scores = metrics.get_knn_matching(gl_dist)
glmatch = [np.arange(glx.shape[0]),gl_full_match,gl_scores]

In [39]:
order = (2, 1)
acc_ann_ms = metrics.get_matching_acc(
        matching=msmatch, 
        labels1=annt, 
        labels2=annt,
        order = order
    )
acc_ann_ms

0.7887788778877888

In [40]:
order = (2, 1)
acc_ann_scj = metrics.get_matching_acc(
        matching=scjmatch, 
        labels1=annt, 
        labels2=annt,
        order = order
    )
acc_ann_scj

0.4444444444444444

In [53]:
order = (2, 1)
acc_ann_mf = metrics.get_matching_acc(
        matching=mfmatch, 
        labels1=annt, 
        labels2=annt,
        order = order
    )
acc_ann_mf

0.9053001758352173

In [45]:
order = (2, 1)
acc_ann_gl = metrics.get_matching_acc(
        matching=glmatch, 
        labels1=annt, 
        labels2=annt,
        order = order
    )
acc_ann_gl

0.9327689243027888

In [54]:
ann_listlv1 = []
ann_listlv1.extend([acc_ann_mf, acc_ann_gl, acc_ann_scj, acc_ann_ms])
ann_listlv1

[0.9053001758352173,
 0.9327689243027888,
 0.4444444444444444,
 0.7887788778877888]

## other metrics

In [69]:
mfx = pd.read_csv("/atac_bench_nrz/10x_e18/mf/full_embed_x0.csv")
mfy = pd.read_csv("/atac_bench_nrz/10x_e18/mf/full_embed_y0.csv")
mfx = mfx.iloc[:,:20]
mfy = mfy.iloc[:,:20]
mf_dist = utils.cdist_correlation(mfx.to_numpy(), mfy.to_numpy())

In [71]:
mf_fos = metrics.get_foscttm(mf_dist)
scj_fos = metrics.get_foscttm(scj_dist)
ms_fos = metrics.get_foscttm(ms_dist)
gl_fos = metrics.get_foscttm(gl_dist)

In [72]:
foscttm_list = []
foscttm_list.extend([mf_fos, gl_fos, scj_fos, ms_fos])
foscttm_list

[0.024018887505149588,
 0.012706676734584732,
 0.37775111255984134,
 0.11861403207561967]

## int related

In [59]:
slt_f1 = []
ari_f1 = []

mf_int = pd.read_csv("/atac_bench_nrz/10x_e18/mf/metrics.csv")
ms_int = pd.read_csv("/atac_bench_nrz/10x_e18/maestro/metrics.csv")
gl_int = pd.read_csv("/atac_bench_nrz/10x_e18/glue/metrics.csv")
scj_int = pd.read_csv("/atac_bench_nrz/10x_e18/scjoint/metrics.csv")

slt_f1.extend([mf_int.loc[0,'slt_f1'], gl_int.loc[0,'slt_f1'], scj_int.loc[0,'slt_f1'], ms_int.loc[0,'slt_f1']])
ari_f1.extend([mf_int.loc[0,'ari_f1'], gl_int.loc[0,'ari_f1'], scj_int.loc[0,'ari_f1'], ms_int.loc[0,'ari_f1']])

slt_f1

[0.5215440606079728,
 0.5290452323497566,
 0.3807741076310094,
 0.5086121768999639]

In [60]:
ari_f1

[0.553539509128713, 0.5730017418732254, 0.4783569836880859, 0.5513785924583343]

In [61]:
m = ["mf","gl","scj","ms"]
minfo = m

data = {'method':minfo,'slt_f1': slt_f1, 'ari_f1':ari_f1,
       'ann1':ann_listlv1, 'foscttm':foscttm_list}

matching_result = pd.DataFrame(data)
matching_result

,method,slt_f1,ari_f1,ann1,foscttm
0,mf,0.521544,0.553540,0.905300,0.028781
1,gl,0.529045,0.573002,0.932769,0.012707
2,scj,0.380774,0.478357,0.444444,0.377751
3,ms,0.508612,0.551379,0.788779,0.118614


In [76]:
matching_result.to_csv("/atac_bench_nrz/10x_e18/metric_results.csv")

## KNN

In [73]:
# knn search tmp
k = 100
knn_match_mf = metrics.get_knn_alignment_score(
    dist=mf_dist,
    k_max=k
)
knn_match_scj = metrics.get_knn_alignment_score(
    dist=scj_dist,
    k_max=k
)
knn_match_ms = metrics.get_knn_alignment_score(
    dist=ms_dist,
    k_max=k
)
knn_match_gl = metrics.get_knn_alignment_score(
    dist=gl_dist,
    k_max=k
)

In [74]:
data = {'mf': knn_match_mf,
        'scj':knn_match_scj,
        'ms':knn_match_ms,
        'gl':knn_match_gl,
       }
knn_tmp = pd.DataFrame(data)
knn_tmp

,mf,scj,ms,gl
0,0.036925,0.000502,0.008289,0.057021
1,0.061291,0.000754,0.014569,0.102989
2,0.083899,0.001507,0.023863,0.134891
3,0.105501,0.002010,0.028636,0.168299
4,0.127355,0.002261,0.036925,0.200452
...,...,...,...,...
95,0.734238,0.039689,0.377543,0.865109
96,0.738006,0.039940,0.380306,0.869380
97,0.740266,0.040191,0.383572,0.870887
98,0.743783,0.040191,0.384326,0.872896


In [77]:
import numpy as np

knn_tmp = []
knn_tmp.extend([knn_match_mf.tolist(), knn_match_gl.tolist(),
                knn_match_scj.tolist(), knn_match_ms.tolist()])
m = np.array(["mf", "gl","scj","ms"])
minfo = np.repeat(m, [100,100,100,100], axis=0).tolist()
minfo2 = minfo #* 5
knn = [item for sublist in knn_tmp for item in sublist]
step = [i for i in range(100)]*4 #*5
data = {'method':minfo2,'knn_tmp': knn, 'step':step }

knn_result = pd.DataFrame(data)
knn_result

,method,knn_tmp,step
0,mf,0.036925,0
1,mf,0.061291,1
2,mf,0.083899,2
3,mf,0.105501,3
4,mf,0.127355,4
...,...,...,...
395,ms,0.377543,95
396,ms,0.380306,96
397,ms,0.383572,97
398,ms,0.384326,98


In [78]:
# save the results as csvs
knn_result.to_csv("/atac_bench_nrz/10x_e18/knntmp.csv")